# ⚡ Energía y Recursos Naturales: continuidad, costos y seguridad
KPIs: factor de planta, OEE, pérdidas no técnicas, incidentes.

## Casos de uso
- Mantenimiento predictivo en equipos críticos
- Balance energético y detección de pérdidas
- Optimización de rutas/logística en upstream/downstream
- Monitoreo ambiental y cumplimiento regulatorio

In [7]:
# Esbozo de alerta por sensor
def alerta(temp, umbral=80):
    return 'ALERTA' if temp>umbral else 'OK'
[alerta(t) for t in [65, 78, 85]]

['OK', 'OK', 'ALERTA']

## Arquitectura
- Ingesta IoT/SCADA streaming, almacenamiento time-series
- Feature engineering en ventana y gemelos digitales
- Integración con sistemas de seguridad y HSE

## 💼 Valor en Energía/Recursos
- Menos paradas no programadas (uptime ↑) y OEE ↑.
- Pérdidas técnicas/no técnicas ↓ con monitoreo y balance energético.
- Cumplimiento HSE y ambiental con datos trazables.


In [8]:
# Práctica: sensor y alerta (toy)
import pandas as pd
import numpy as np
np.random.seed(0)
serie = pd.Series(70 + np.random.randn(50)*5)
movil = serie.rolling(5).mean()
alertas = (serie > 80).sum()
print('Temp promedio:', round(serie.mean(),1))
print('Ventana(5) promedio última:', round(movil.iloc[-1],1))
print('Alertas >80°C:', int(alertas))


Temp promedio: 70.7
Ventana(5) promedio última: 67.3
Alertas >80°C: 2


## 🏗️ Arquitectura IoT/Time-Series
- Ingesta SCADA/IoT en streaming, almacenamiento TSDB/lakehouse.
- Feature engineering en ventana, gemelos digitales y mantenimiento predictivo.
- Integración con HSE y sistemas de seguridad.


In [9]:
# Visual: serie de temperatura y umbral
import plotly.express as px
import pandas as pd
import numpy as np
serie_df = pd.DataFrame({'t': np.arange(len(serie)), 'temp': serie})
fig = px.line(serie_df, x='t', y='temp', title='Temperatura de sensor')
fig.add_hline(y=80, line_dash='dash', line_color='red')
fig.show()


## ✅ Checklist de estrategia de negocio y datos
- Objetivo (uptime/OEE, costos, seguridad): …
- KPIs: OEE, pérdidas técnicas/no técnicas, incidentes.
- Palancas: IoT/SCADA, series de tiempo, mantenimiento predictivo.
- Datos/calidad: señales, muestreo, tests en ventana.
- Gobierno: dominios, compliance HSE.
- SLOs: latencia/alertas, costo, disponibilidad.
- ROI/supuestos: …
- Riesgos: seguridad física/cibernética, calidad sensor.


## 🔗 Puente Estrategia ↔ Ingeniería de Datos
- OKR: OEE ≥ 0.90 y incidentes 0 → KPI negocio: OEE, incidentes → KPI datos: disponibilidad sensores, latencia alertas → Capacidades: IoT/SCADA + TSDB + monitoreo → Decisión: mantenimiento preventivo → Impacto: menos paradas.

```
[Sensores]→[Ingesta IoT]→[Features ventana]→[OEE/Alertas]→[Umbral]→[Mantenimiento]→[Δ downtime]
```


In [10]:
# contrato_de_datos (energía)
contrato_datos = {
    "objetivo_negocio": "OEE ≥ 0.90",
    "kpi_negocio": "OEE promedio",
    "kpi_datos": {"uptime_sensores_%": ">=99%", "latencia_alerta_s": "<10"},
    "dataset": ["telemetria", "ordenes_mto"],
    "propietario": "Dominio Operaciones",
    "slo": {"retencion_ts_dias": ">=30"},
    "decision": "Mantenimiento preventivo semanal",
    "impacto_economico_usd": 220000,
}
print(contrato_datos)


{'objetivo_negocio': 'OEE ≥ 0.90', 'kpi_negocio': 'OEE promedio', 'kpi_datos': {'uptime_sensores_%': '>=99%', 'latencia_alerta_s': '<10'}, 'dataset': ['telemetria', 'ordenes_mto'], 'propietario': 'Dominio Operaciones', 'slo': {'retencion_ts_dias': '>=30'}, 'decision': 'Mantenimiento preventivo semanal', 'impacto_economico_usd': 220000}


## 📋 Caso de uso: Mantenimiento predictivo con IoT y time-series

**Contexto de negocio:** Una planta de generación eléctrica tiene paradas no programadas que cuestan $500k/día. Objetivo: OEE ≥ 0.90 y cero incidentes HSE.

**Problema sin ingeniería de datos:**
- Sensores SCADA guardan datos localmente sin integración.
- Alarmas manuales basadas en umbrales fijos (muchos falsos positivos).
- Mantenimiento reactivo: se actúa cuando ya falló el equipo.

**Solución con ingeniería de datos:**
1. Ingesta IoT/SCADA en streaming: 10k señales/seg → TSDB (InfluxDB/TimescaleDB).
2. Feature engineering en ventana: media móvil, desviación, tasa de cambio.
3. Modelo de gemelo digital: predice fallo 48h antes con 85% de precisión.
4. Alertas accionables: "Bomba B03 - riesgo alto de cavitación en 36h".
5. Integración con CMMS (sistema de mantenimiento) para generar órdenes automáticas.

**Resultado:**
- OEE: 0.82 → 0.91.
- Paradas no programadas: 12/año → 2/año.
- Ahorro anual: $4.5M (menos downtime + mantenimiento optimizado).
- Cumplimiento HSE: 100% trazabilidad de eventos críticos.


In [11]:
# Ejercicio: detección de anomalías en series de tiempo
import pandas as pd
import numpy as np

# Simular telemetría de sensor (temperatura)
np.random.seed(0)
tiempo = pd.date_range('2025-01-01', periods=100, freq='h')
temp_base = 70 + np.random.randn(100) * 3
# Inyectar anomalía
temp_base[50:55] = 95

datos_sensor = pd.DataFrame({'timestamp': tiempo, 'temp': temp_base})

# Feature engineering: usar media histórica (excluye valor actual)
# Método 1: Umbral fijo basado en conocimiento del dominio
umbral_fijo = 80  # Temperatura operativa normal < 80°C
datos_sensor['anomalia_fija'] = datos_sensor['temp'] > umbral_fijo

# Método 2: Detección por desviación respecto a baseline (30 primeras horas = periodo normal)
baseline_mean = datos_sensor['temp'].iloc[:30].mean()
baseline_std = datos_sensor['temp'].iloc[:30].std()
datos_sensor['umbral_dinamico'] = baseline_mean + 3 * baseline_std
datos_sensor['anomalia_dinamica'] = datos_sensor['temp'] > datos_sensor['umbral_dinamico']

# Combinar ambos métodos
datos_sensor['anomalia'] = datos_sensor['anomalia_fija'] | datos_sensor['anomalia_dinamica']

anomalias = datos_sensor[datos_sensor['anomalia'] == True]
print(f"Anomalías detectadas: {len(anomalias)}")
print(f"  - Por umbral fijo (>80°C): {datos_sensor['anomalia_fija'].sum()}")
print(f"  - Por desviación (>3σ de baseline): {datos_sensor['anomalia_dinamica'].sum()}")
print("\nPrimeras anomalías:")
print(anomalias[['timestamp', 'temp', 'umbral_dinamico']].head())
print(f"\n💡 Pipeline streaming puede generar alerta en <10s para intervención")
print(f"⚠️ Temperatura crítica detectada: {datos_sensor['temp'].max():.1f}°C (límite: {umbral_fijo}°C)")

Anomalías detectadas: 5
  - Por umbral fijo (>80°C): 5
  - Por desviación (>3σ de baseline): 5

Primeras anomalías:
             timestamp  temp  umbral_dinamico
50 2025-01-03 02:00:00  95.0        81.231525
51 2025-01-03 03:00:00  95.0        81.231525
52 2025-01-03 04:00:00  95.0        81.231525
53 2025-01-03 05:00:00  95.0        81.231525
54 2025-01-03 06:00:00  95.0        81.231525

💡 Pipeline streaming puede generar alerta en <10s para intervención
⚠️ Temperatura crítica detectada: 95.0°C (límite: 80°C)


---

## 🧭 Navegación

**← Anterior:** [🏥 Salud y Farmacéutico: calidad, trazabilidad, acceso](04_salud_farmaceutico.ipynb)

**Siguiente →:** [📶 Telecomunicaciones: churn, ARPU y calidad de red →](06_telecomunicaciones.ipynb)

**📚 Índice de Negocio LATAM:**
- [🎯 Estrategia de Datos en LATAM: de la visión al ROI](01_estrategia_datos_latam.ipynb)
- [🛒 Retail y Consumo Masivo: demanda, surtido y pérdidas](02_retail_consumo_masivo.ipynb)
- [💳 Finanzas y Banca: riesgo, fraude, rentabilidad](03_finanzas_banca.ipynb)
- [🏥 Salud y Farmacéutico: calidad, trazabilidad, acceso](04_salud_farmaceutico.ipynb)
- [⚡ Energía y Recursos Naturales: continuidad, costos y seguridad](05_energia_recursos_naturales.ipynb) ← 🔵 Estás aquí
- [📶 Telecomunicaciones: churn, ARPU y calidad de red](06_telecomunicaciones.ipynb)
- [🏭 Industria y Manufactura: OEE, calidad y throughput](07_industria_manufactura.ipynb)
- [🚚 Logística y Transporte: costo por entrega, SLA, ocupación](08_logistica_transporte.ipynb)
- [🌾 Agro y Alimentos: rendimiento, calidad y trazabilidad](09_agro_alimentos.ipynb)
- [🏛️ Sector Público y Gobierno: transparencia, servicio y eficiencia](10_sector_publico_gobierno.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
